In [14]:
import pandas as pd
from data.movielens_data import MovielensData
from data.steam_data import SteamData
#from goodreads_data import GoodreadsData
from data.lastfm_data import LastfmData
import json
from tqdm import tqdm
from recommender.A_SASRec_final_bce_llm import SASRec

In [7]:
cans_num = 20
data_dir = "./data/ref/lastfm"
lastfm = LastfmData(data_dir=data_dir, stage='train', cans_num=cans_num)

In [20]:
id2name = lastfm.get_game_id2name()
name2id = {v: k for k, v in id2name.items()}

In [21]:
name2id

{'Morcheeba': 0,
 'Enigma': 1,
 'Café Del Mar': 2,
 'Fleetwood Mac': 3,
 'China Crisis': 4,
 'Loscil': 5,
 'Chicane': 6,
 'Sigue Sigue Sputnik': 7,
 'Duran Duran': 8,
 'Air': 9,
 'Röyksopp': 10,
 'Moby': 11,
 'Depeche Mode': 12,
 'Groove Armada': 13,
 'INXS': 14,
 'Deep Forest': 15,
 'Porcupine Tree': 16,
 'De/Vision': 17,
 'Radiohead': 18,
 'VAST': 19,
 'Michael Jackson': 20,
 'God Is an Astronaut': 21,
 'Pink Floyd': 22,
 'Planet Funk': 23,
 'Scissor Sisters': 24,
 'Mew': 25,
 'Stereophonics': 26,
 'Placebo': 27,
 'Infected Mushroom': 28,
 'Delerium': 29,
 'Roxette': 30,
 'Paradise Lost': 31,
 'Jamiroquai': 32,
 'James Blunt': 33,
 'Reamonn': 34,
 'Opeth': 35,
 'Elton John': 36,
 'Tears for Fears': 37,
 'Death Cab for Cutie': 38,
 'Savage Garden': 39,
 'Darren Hayes': 40,
 'Explosions in the Sky': 41,
 'AC/DC': 42,
 'Apocalyptica': 43,
 'Megadeth': 44,
 'Dream Theater': 45,
 'Tiësto': 46,
 'The Darkness': 47,
 'VNV Nation': 48,
 'Thirteen Senses': 49,
 'Dave Gahan': 50,
 'Apparat': 5

In [3]:
lastfm[0]

{'seq': [0, 1, 2, 4606, 4606, 4606, 4606, 4606, 4606, 4606],
 'seq_name': ['Morcheeba', 'Enigma', 'Café Del Mar'],
 'len_seq': 3,
 'seq_str': 'Morcheeba, Enigma, Café Del Mar',
 'cans': [1731,
  2064,
  2366,
  2056,
  2617,
  1033,
  1225,
  3156,
  3955,
  4342,
  3,
  3712,
  3566,
  1288,
  786,
  253,
  1616,
  2485,
  52,
  1365],
 'cans_name': ['Throbbing Gristle',
  'Chimarruts',
  'The Almost',
  'Lordi',
  'Antony and the Johnsons',
  'Alcazar',
  'Ace of Base',
  'Hellogoodbye',
  'Collective Soul',
  'Everclear',
  'Fleetwood Mac',
  'Ian Brown',
  'Entwine',
  'Beck',
  'Cinema Strange',
  'Tracktor Bowling',
  'Gary Moore',
  'Lisa Hannigan',
  'Def Leppard',
  'Hannah Montana'],
 'cans_str': 'Throbbing Gristle, Chimarruts, The Almost, Lordi, Antony and the Johnsons, Alcazar, Ace of Base, Hellogoodbye, Collective Soul, Everclear, Fleetwood Mac, Ian Brown, Entwine, Beck, Cinema Strange, Tracktor Bowling, Gary Moore, Lisa Hannigan, Def Leppard, Hannah Montana',
 'len_cans':

In [25]:
print(lastfm[0]["seq"])
print(lastfm[0]["cans"])
# print(lastfm[0]["item_id"])

seq = lastfm[0]["seq"]
cans = lastfm[0]["cans"]
# item_id = lastfm[0]["item_id"]

tensor([   0,    1,    2, 4606, 4606, 4606, 4606, 4606, 4606, 4606])
tensor([2128, 3034,  111,  540, 3906, 3257, 1072, 3856, 1953, 4462, 4435, 1923,
           3, 4516, 4506, 1574, 3887, 1237, 1903, 3847])


In [74]:
negative_items = lastfm[0]["cans_name"]

id2name = lastfm.get_game_id2name()
name2id = {v: k for k, v in id2name.items()}
neg_seq_ids = [name2id[i] for i in negative_items]
neg_seq_ids_tensor = torch.tensor(neg_seq_ids).unsqueeze(0).cuda(0)
rec_model_path = "./rec_model/lastfm.pt"
sasrec_model = torch.load(rec_model_path, map_location="cuda:0")
sasrec_model.eval()
for name, param in sasrec_model.named_parameters():
    param.requires_grad = False

len(neg_seq_ids)
len(neg_seq_ids_tensor)
scores = sasrec_model.forward_eval(neg_seq_ids_tensor[:,:10], torch.tensor(10).reshape(-1).cuda(0))
sorted_indices = torch.argsort(scores)
print(sorted_indices)
print(neg_seq_ids)
print(neg_seq_ids[sorted_indices])


tensor([1390,  923, 1509,  ..., 3309, 2545, 3888], device='cuda:0')
[2128, 3034, 111, 540, 3906, 3257, 1072, 3856, 1953, 4462, 4435, 1923, 3, 4516, 4506, 1574, 3887, 1237, 1903, 3847]


TypeError: only integer tensors of a single element can be converted to an index

In [33]:
import torch
rec_model_path = "./rec_model/lastfm.pt"
sasrec_model = torch.load(rec_model_path, map_location="cuda")
#self.rec_model = torch.load(rec_model_path, map_location="cpu")

In [34]:
sasrec_model.eval()
for name, param in sasrec_model.named_parameters():
    param.requires_grad = False

In [35]:
sasrec_model

SASRec(
  (dropout): Dropout(p=0.1, inplace=False)
  (item_embeddings): Embedding(4607, 64)
  (positional_embeddings): Embedding(10, 64)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (ln_3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (mh_attn): MultiHeadAttention(
    (linear_q): Linear(in_features=64, out_features=64, bias=True)
    (linear_k): Linear(in_features=64, out_features=64, bias=True)
    (linear_v): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (softmax): Softmax(dim=-1)
  )
  (feed_forward): PositionwiseFeedForward(
    (w_1): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (w_2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (s_fc): Linear(in_features=64, out

In [37]:
sasrec_model.cacu_x(torch.tensor(seq).cuda())

/tmp/ipykernel_1128244/3234804940.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sasrec_model.cacu_x(torch.tensor(seq).cuda())


tensor([[ 3.9008e-02, -3.7917e-03,  3.8274e-02, -1.0216e-02, -1.9958e-02,
         -9.7198e-03, -1.9146e-02, -3.7327e-02, -7.1490e-02,  1.0672e-02,
         -7.9374e-03, -1.0039e-02,  2.2457e-02, -9.4023e-02,  9.9973e-03,
          1.7599e-02, -4.3048e-03, -1.6548e-02, -1.4033e-01,  5.8583e-02,
         -1.5488e-02,  7.7659e-03,  1.7219e-02, -3.1946e-02, -1.2858e-02,
          2.3826e-02, -4.0800e-02, -1.3066e-02,  8.2931e-03,  1.0917e-03,
         -1.7102e-02,  5.9709e-02,  2.8146e-02, -1.2288e-02, -1.3526e-02,
         -7.5823e-02, -4.6233e-02,  1.1287e-01, -3.3210e-02, -8.1306e-03,
          3.3449e-02, -3.0960e-02,  6.7672e-02, -4.3561e-02, -2.5907e-02,
          5.3993e-02,  6.3605e-02,  1.1033e-01, -2.0479e-02,  2.0234e-02,
          2.5217e-02, -9.0505e-02,  3.6572e-02, -5.5015e-02, -2.1023e-02,
         -3.1004e-02, -1.5263e-02,  2.3160e-02, -2.7352e-02, -6.1957e-03,
         -1.1219e-02, -6.0915e-03, -2.2118e-02,  3.9385e-03],
        [-4.7875e-02,  1.3284e-02, -5.4048e-02,  2

In [41]:
seq_len_states =torch.tensor(len(seq)).reshape(1,1).cuda()

In [46]:
results = sasrec_model.forward_eval(torch.tensor(seq).reshape(1, -1).cuda(), seq_len_states)

/tmp/ipykernel_1128244/2498548871.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  results = sasrec_model.forward_eval(torch.tensor(seq).reshape(1, -1).cuda(), seq_len_states)


In [48]:
results.shape # 返回对所有item的打分

torch.Size([4606])

In [50]:
results[0: 50]

tensor([ 1.9952,  1.5371,  0.1817,  0.9581, -0.5323, -0.7945, -0.9347, -0.8378,
         2.8566,  2.5586,  1.5609,  0.8648,  2.2864,  0.6515,  0.5770,  0.6925,
         1.7054, -0.2849,  2.1750,  0.3075,  1.8450,  1.2597,  1.9991,  0.3529,
         0.6873,  0.8396,  0.7758,  1.7580,  0.3019,  0.0331,  0.7867,  0.1837,
         0.9042,  0.8285, -0.1515,  1.4790,  0.4344,  0.1662,  1.3542, -0.4542,
        -0.5213, -0.7330, -0.1048, -0.5754, -0.3335, -0.5431, -0.5322, -0.8950,
        -0.8245, -0.7180], device='cuda:0')